In [1]:
import gzip
import json
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from torch_geometric.utils.convert import from_scipy_sparse_matrix
import torch
from pandas import json_normalize
from IPython.display import display

In [2]:
print(torch.cuda.is_available())
device = torch.device('cuda')

False


In [159]:
with gzip.open('../data/RosettaStone-GraphData-2023-09-22/superblue1/1/superblue1.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))
    
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('../data/RosettaStone-GraphData-2023-09-22/superblue1/superblue1_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])

with gzip.open('../data/RosettaStone-GraphData-2023-09-22/cells.json.gz', 'rb') as f:
    cells_json = json.loads(f.read())
cells = pd.DataFrame(cells_json).drop(columns=['name', 'width', 'height']).set_index('id')

In [160]:
cells['terms'] = cells['terms'].apply(lambda x: pd.DataFrame(x))
cells

,terms
id,
0,name id dir xloc yloc 0 A 1 0 ...
1,name id dir xloc yloc 0 A 1 0 ...
2,name id dir xloc yloc 0 A 1 0 ...
3,name id dir xloc yloc 0 D 1 0...
4,name id dir xloc yloc 0 A 1 0 ...
...,...
30131,Empty DataFrame Columns: [] Index: []
30132,Empty DataFrame Columns: [] Index: []
30133,Empty DataFrame Columns: [] Index: []


In [161]:
instances.head()

,name,id,xloc,yloc,cell,orient
0,o0,0,3567360,5044950,132,0
1,o1,1,2863200,5351310,359,6
2,o10,2,2846400,5351310,176,0
3,o100,3,2508000,5744250,224,0
4,o1000,4,2479680,5757570,360,0


In [162]:
A_tensor = torch.sparse_coo_tensor(
    torch.FloatTensor([A.row, A.col]),
    torch.Tensor(A.data)
).coalesce()
#A_tensor.to(device)

In [163]:
A_tensor.indices()

tensor([[     0,      0,      0,  ..., 797937, 797937, 797937],
        [613721, 613832, 613943,  ...,      0, 756354, 822927]])

In [164]:
A_tensor.values().unique()

tensor([1.0000e+00, 2.0000e+00, 3.0000e+00,  ..., 1.2410e+03, 1.2420e+03,
        1.2430e+03])

In [165]:
cells['terms'].iloc[1243]#.iloc[0]

,name,id,dir,xloc,yloc
0,p0,1,1,172800,28120
1,p1,2,0,67200,29600


In [166]:
def direct(c, val):
    '''
    params:
    c: column index / terminal
    val: tensor value for lookup
    '''
    try:
        direction = cells['terms'][instances.iloc[c]['cell']].loc[val]['dir']
        if direction == 1:
            return 0
        return 1
    except:
        print(c, val)

In [156]:
# A_tensor.indices()[1] is columns of terminal matrix, or gates
indices = A_tensor.indices()
terminal_values = (A_tensor.values() - 1).clone()
gate_indices = indices[0]

In [167]:
i = 0
for gate_index, val in zip(gate_indices, terminal_values):
    if i % 100000 == 0:
        print(i)
    i += 1
    A_tensor.values()[gate_index.item()] = direct(c=gate_index.item(), val=val.item())

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
732233 54.0


TypeError: can't assign a NoneType to a torch.FloatTensor

In [168]:
cells['terms'][instances.iloc[732233]['cell']]

,name,id,dir,xloc,yloc
0,p0,1,1,9600,5920
1,p1,2,1,9600,85840
2,p2,3,1,9600,254560
3,p3,4,1,9600,113960
4,p4,5,1,9600,281200
5,p5,6,1,9600,226440
6,p6,7,1,9600,32560
7,p7,8,1,9600,59200
8,p8,9,1,9600,199800
9,p9,10,1,9600,213120
